In [1]:
from dotenv import load_dotenv
import os
import fitz  # PyMuPDF
from langchain.embeddings import OpenAIEmbeddings
from langchain_objectbox.vectorstores import ObjectBox ##vector Database
from langchain.chains import RetrievalQA
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain import hub
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain import hub
from langchain.docstore.document import Document

In [2]:
# Load environment variables
load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

In [3]:
# Function to extract text from a PDF file
def extract_text_from_pdf(pdf_path):
    pdf_document = fitz.open(pdf_path)
    text = ""
    for page_num in range(len(pdf_document)):
        page = pdf_document.load_page(page_num)
        text += page.get_text()
    return text

In [4]:
# Function to split text into smaller chunks
def split_text(text, max_chunk_size=1000):
    words = text.split()
    chunks = []
    chunk = []
    chunk_size = 0
    for word in words:
        if chunk_size + len(word) + 1 <= max_chunk_size:
            chunk.append(word)
            chunk_size += len(word) + 1
        else:
            chunks.append(" ".join(chunk))
            chunk = [word]
            chunk_size = len(word) + 1
    if chunk:
        chunks.append(" ".join(chunk))
    return chunks

In [5]:
# Extract data from local PDF file
pdf_path = '/Users/abhishekshah/Desktop/rag/human-nutrition-text.pdf'
document_content = extract_text_from_pdf(pdf_path)
document_chunks = split_text(document_content)

# Create Document objects for each chunk
documents = [Document(page_content=chunk) for chunk in document_chunks]

In [6]:
# Create vector embeddings
embeddings = OpenAIEmbeddings()
vector = ObjectBox.from_documents(documents, embeddings, embedding_dimensions=768)

/Users/abhishekshah/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [7]:
# Make a RAG pipeline
llm = ChatOpenAI(model="gpt-4o")
prompt = hub.pull("rlm/rag-prompt")

# Build a retriever
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vector.as_retriever(),
    chain_type_kwargs={"prompt": prompt}
)

In [8]:
# Ask a question
question = "what are complex carbs?"
result = qa_chain({"query": question})
print(result["result"])

/Users/abhishekshah/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Complex carbohydrates are long chains of monosaccharides that may be branched or unbranched. They include polysaccharides such as starches and fibers. Examples of foods rich in complex carbs are grains, legumes, and root vegetables like potatoes.
